In [ ]:
import re
from bs4 import BeautifulSoup
from google.colab import drive
from tensorflow.keras.models import Sequential
import pandas as pd


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cols=['sentiment','id','date','query','user','text']
df=pd.read_csv("/content/drive/MyDrive/Projects/Twitter_sentiment_analysis/training.1600000.processed.noemoticon.csv",
               engine='python',
               header=None,
               names=cols,
               encoding="Latin1")
#   Latin1 encoding because it accept any possible byte as input, convert into unicode character

df.drop(['id','date','query','user'],axis=1,inplace=True)
df.tail()

,sentiment,text
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,happy #charitytuesday @theNSPCC @SparksCharity...


In [ ]:
def clean_data(tweet):
  tweet = BeautifulSoup(tweet,"lxml").get_text()
  #  "<html><head><title>test<body><h1>page title</h3>" ----> test page title
  
  tweet = re.sub(r"@[a-zA-Z0-9]+",'',tweet)
  #  "@hello i am good" -----> "i am good"
  
  tweet = re.sub(r"https?://[a-zA-Z0-9./]+",'',tweet)
  #  "https://zindi.africa/competitions" -----> ""
  
  tweet = re.sub(r"[^a-zA-Z.!?']+"," ",tweet)
  #   "*hello i am good 900" -----> hollo i am good
  
  tweet = re.sub(r" +"," ",tweet)
  #   double spaces are removed
  return tweet

In [ ]:
cleaned_data=[clean_data(tweet) for tweet in df.text]

In [ ]:
df_labels=df.sentiment.values
df_labels[df_labels==4]=1

In [ ]:
cleaned_data[0:2]

[" Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D",
 "is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!"]

In [ ]:

vocab_size = 2000000
embedding_dim = 16
max_length = 200
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cleaned_data, df_labels, test_size=0.30, random_state=42)

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
# num_words is no. of words it should take into account. and num_words should ve as much as possible (without stop words)

tokenizer.fit_on_texts(cleaned_data)
word_index = tokenizer.word_index
#  "The cat sat on the mat."  -----> {'cat': 2, 'mat': 5, 'on': 4, 'sat': 3, 'the': 1}
# lesser the word_index value, it has occured frequently in the context

training_sequences = tokenizer.texts_to_sequences(X_train)
# values are assigned in place of words based on the word index
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
# trunc removing extra words
# adding "0"s to sequence

testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
#We store this tokenizer in a file to use later in web app
import pickle
# saving
with open('/content/drive/MyDrive/Projects/Twitter sentiment analysis/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPool1D, Dense, Dropout

In [ ]:
VOCAB_SIZE =vocab_size
EMB_DIM = 200
nb_filters  = 50  
FFN_units = 200
NB_CLASSES = 2
dropout_rate = 0.2
BATCH_SIZE = 32
NB_EPOCHS = 2

In [ ]:
model1 = Sequential()
#  it does not allow you to create models that share layers or have multiple inputs or outputs

model1.add(tf.keras.layers.Embedding( vocab_size, embedding_dim, input_length=max_length))
# Word embeddings provide a dense representation of words and their relative meanings.
# input_dim: This is the size of the vocabulary in the text data.
# embeding_dim : a vector space of 16 dimensions
# input_length : length of each input document 

model1.add(Conv1D(filters=nb_filters,
                                kernel_size=2,
                                padding = 'valid',
                                activation = "relu"))
# "valid" applies padding to the input sequence so the output size shrinks by filter_size - 1. No padding occurs.
# relu because return positive values and negative values as zero.
model1.add(Conv1D(filters = nb_filters,
                                 kernel_size = 3,
                                 padding = "valid",
                                 activation = "relu"))
model1.add(Conv1D(filters = nb_filters,
                                 kernel_size = 4,
                                 padding = "valid",
                                activation = 'relu'))
model1.add(GlobalMaxPool1D())
# Downsamples the input representation by taking the maximum value over the time dimension.
# shape = input_length,input_channel

model1.add(Dense(units = FFN_units,activation = "relu"))
model1.add(Dropout(rate = dropout_rate))
# dropout layer is to prevent overfitting.

model1.add(Dense(1, activation='sigmoid'))
# "sigmoid" because we want output ranging between 0 and 1


In [ ]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 16)           32000000  
_________________________________________________________________
conv1d (Conv1D)              (None, 199, 50)           1650      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 197, 50)           7550      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 194, 50)           10050     
_________________________________________________________________
global_max_pooling1d (Global (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 200)               10200     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0

In [ ]:
model1.compile(loss = "binary_crossentropy",
               optimizer = 'adam',
               metrics = ['accuracy'])

# binary_crossentropy because its a binary__classificartion problem (  -(1/N) sum(y * log(yhat) + (1 -y) * log(1 - yhat)) )
# optimizer deals with learning rate and when should update weight.

In [ ]:

checkpoint_path = "/content/drive/MyDrive/Projects/Twitter sentiment analysis/checkpoint/"

ckpt = tf.train.Checkpoint(model1)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [ ]:
model1.fit(training_padded,y_train,validation_data= (testing_padded,y_test),batch_size = 2**8, epochs = NB_EPOCHS, verbose =1) 
ckpt_manager.save()

Epoch 1/2
4375/4375 [==============================] - 726s 166ms/step - loss: 0.4262 - accuracy: 0.8021 - val_loss: 0.3939 - val_accuracy: 0.8214
Epoch 2/2
4375/4375 [==============================] - 725s 166ms/step - loss: 0.3610 - accuracy: 0.8406 - val_loss: 0.3956 - val_accuracy: 0.8227


'/content/drive/MyDrive/Projects/Twitter sentiment analysis/checkpoint/ckpt-1'

In [ ]:
predict_1 = model1.predict(testing_padded,batch_size = 2**8,verbose=1)

1875/1875 [==============================] - 4s 2ms/step


In [ ]:
ori = lambda x:0 if x<0.5 else 1

In [ ]:
predict_1_y = list(map(ori,predict_1))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predict_1_y))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82    239361
           1       0.83      0.81      0.82    240639

    accuracy                           0.82    480000
   macro avg       0.82      0.82      0.82    480000
weighted avg       0.82      0.82      0.82    480000



In [ ]:
sen = ["i love him"]
tokenizer=tfds.deprecated.text.Tokenizer()
input_data1 = [tokenizer.tokenize(i) for i in sen]

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
sen = tokenizer.texts_to_sequences(input_data1)
#sen= pad_sequences(sen, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
vocab_size

1120000

In [ ]:
reviews = ['they made me cry', 'I hate spaghetti',
           "he couldn't make it", 
                'Everything was good',
                'he is a theif', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
              "everyone was happy "]

ori = lambda x:0 if x<0.5 else 1
# Create the sequences
padding_type='post'
sample_data=[clean_data(tweet) for tweet in reviews]
#sample_stop = [stop_remove(i) for i in sample_data]
sample_sequences = tokenizer.texts_to_sequences(sample_data)
reviews_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           
classes = model1.predict(reviews_padded)
classes_y=list( map(ori,classes))
for i in range(len(reviews)):
  print(reviews[i],classes_y[i])


they made me cry 0
I hate spaghetti 0
he couldn't make it 0
Everything was good 1
he is a theif 1
Everything was green 1
the host seated us immediately 1
they gave us free chocolate cake 1
not sure about the wilted flowers on the table 1
only works when I stand on tippy toes 0
everyone was happy  1


In [ ]:
model1.save("/content/drive/MyDrive/Projects/Twitter sentiment analysis/sentient_model1.h5")

**For web app**

In [ ]:
#!pip install streamlit

In [ ]:
import pickle
from tensorflow.keras.models import load_model
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from bs4 import BeautifulSoup
#import streamlit as st

In [ ]:
def clean_data(tweet):
  tweet = BeautifulSoup(tweet,"lxml").get_text()
  tweet = re.sub(r"@[a-zA-Z0-9]+",'',tweet)
  tweet = re.sub(r"https?://[a-zA-Z0-9./]+",'',tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']+"," ",tweet)
  tweet = re.sub(r" +"," ",tweet)

  return tweet

In [ ]:
with open("/content/drive/MyDrive/Projects/Twitter_sentiment_analysis/tokenizer.pickle", "rb") as handle:
 tokenizer = pickle.load(handle)
model=load_model('/content/drive/MyDrive/Projects/Twitter_sentiment_analysis/sentient_model1.h5')

In [ ]:
reviews = ['they made me cry', 'I hate spaghetti',
           "he couldn't make it", 
                'Everything was good',
                'he is a theif', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
              "everyone was happy "]

ori = lambda x:1 if x >0.6 else 0
# Create the sequences
padding_type='post'
sample_data=[clean_data(tweet) for tweet in reviews]
#sample_stop = [stop_remove(i) for i in sample_data]
sample_sequences = tokenizer.texts_to_sequences(sample_data)
reviews_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=200)           
classes = model.predict(reviews_padded)
classes_y=list(map(ori,classes))
for i in range(len(reviews)):
  print(reviews[i],classes_y[i])


they made me cry 0
I hate spaghetti 0
he couldn't make it 0
Everything was good 1
he is a theif 1
Everything was green 1
the host seated us immediately 1
they gave us free chocolate cake 1
not sure about the wilted flowers on the table 1
only works when I stand on tippy toes 0
everyone was happy  1


In [ ]:
def predict(message):
 model=load_model('/content/drive/MyDrive/Projects/Twitter_sentiment_analysis/sentient_model1.h5')
 with open("/content/drive/MyDrive/Projects/Twitter_sentiment_analysis/tokenizer.pickle", "rb") as handle:
  tokenizer = pickle.load(handle)
 x_1 = tokenizer.texts_to_sequences([message])
 x_1 = pad_sequences(x_1, maxlen=500)
 predictions = model.predict(x_1)[0][0]
 ori = 1 if predictions >0.6 else 0
 return (predictions,ori)

In [ ]:
predict("i have to much of workload ")

(0.22874281, 0)